# Benchmarks

## Example 1: Timing comparisons for creating Cardinal BSplines.

In [33]:
import time
from src.nurbs.bsplines import CardinalBSpline

"""
List of B-spline degrees for performance benchmarking.

Selected degrees represent computationally challenging cases where
the caching mechanism provides significant performance benefits:
- Degree 5: Moderate complexity, typical for engineering applications
- Degree 10: High complexity, stress-testing recursive algorithms
- Degree 15: Very high complexity, demonstrating caching scalability

Lower degrees (1-4) are excluded as they execute too quickly to
meaningfully measure caching performance improvements.
"""
degrees = [5, 10, 15]


"""
List storing first-call execution times for each degree.

Each entry corresponds to the time required for initial computation
and compilation of a CardinalBSpline object at the specified degree.
These measurements represent the worst-case scenario without caching.
"""
times_first = []


"""
List storing cached-call execution times for each degree.

Each entry corresponds to the time required to retrieve a pre-computed
CardinalBSpline object from the registry cache. These measurements
demonstrate the optimal performance achieved through memoization.
"""
times_cached = []


print("B-spline Creation Benchmark")
print("=" * 50)

for degree in degrees:
    """
    Reset the class-level registry before each test to ensure
    clean measurement conditions and prevent cache contamination
    from previous iterations.
    """
    CardinalBSpline.registry.clear()
    
    
    """
    Measure time for initial B-spline construction.
    
    This includes:
    - Recursive computation using Cox-de Boor algorithm
    - Polynomial coefficient calculations
    - Function space construction
    - Cache storage operations
    """
    start_time = time.time()
    B_first = CardinalBSpline(m=degree)
    first_time = time.time() - start_time
    
    """
    Measure time for cached B-spline retrieval.
    
    This operation demonstrates the caching efficiency by:
    - Key-based registry lookup
    - Deep copy of cached object
    - Minimal computational overhead
    - O(1) access time complexity
    """
    start_time = time.time()
    B_cached = CardinalBSpline(m=degree)
    cached_time = time.time() - start_time
    
    
    """
    Store timing results for subsequent analysis and visualization.
    The significant difference between first_time and cached_time
    quantifies the performance benefit of the caching system.
    """
    times_first.append(first_time)
    times_cached.append(cached_time)
    
    """
    Print benchmarking results for immediate analysis.
    
    Output includes:
    - Degree being tested
    - First-call execution time in milliseconds
    - Cached-call execution time in milliseconds (typically μs range)
    - Number of objects in registry (verifies caching behavior)
    
    Expected pattern: cached calls should be 10-1000x faster than
    first calls, demonstrating the caching system's effectiveness.
    """
    print(f"Degree {degree}:")
    print(f"  First call: {first_time*1000:6.2f} ms")
    print(f"  Cached call: {cached_time*1000:6.3f} ms")
    print(f"  Cache hits: {len(CardinalBSpline.registry)}")
    

B-spline Creation Benchmark
Degree 5:
  First call:  52.03 ms
  Cached call:  1.002 ms
  Cache hits: 5
Degree 10:
  First call: 191.99 ms
  Cached call:  0.000 ms
  Cache hits: 10
Degree 15:
  First call: 645.82 ms
  Cached call:  0.000 ms
  Cache hits: 15


## Example 2: Timing comparisons for creating NonUniform BSplines.

In [36]:
import time
import numpy as np
from src.nurbs.bsplines import NonUniformBSpline
from src.containers import Knotvector

"""
List of knot vectors with progressively increasing complexity for performance testing.

The test cases are designed to evaluate NonUniformBSpline performance across
different scenarios:
- Simple: Basic uniform knot vector (degree 2)
- Medium: Non-uniform with interior knots (degree 2)  
- Complex: Highly non-uniform with multiple interior knots (degree 3)

This progression tests the caching system's effectiveness with varying
computational loads and knot distribution complexities.
"""
knot_vectors = [
    Knotvector(degree=2, domain=(0, 1), name="simple"),      # Simple uniform
    Knotvector(degree=2, domain=(0, 1), name="medium"),      # Medium complexity
    Knotvector(degree=3, domain=(0, 1), name="complex")      # Higher complexity
]


"""
Modify the medium complexity knot vector to include interior knots.

The knot sequence [0, 0, 0, 0.3, 0.7, 1, 1, 1] creates a cubic B-spline basis
with two interior knots at 0.3 and 0.7, introducing non-uniform spacing
that increases computational complexity compared to uniform knot vectors.
"""
knot_vectors[1].items = np.array([0, 0, 0, 0.3, 0.7, 1, 1, 1])  


"""
Modify the complex knot vector with multiple interior knots.

The knot sequence [0, 0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1, 1] creates a 
cubic B-spline basis with four interior knots, representing a more complex 
case.
"""
knot_vectors[2].items = np.array([0, 0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1, 1]) 


"""
List storing first-call execution times for each knot vector configuration.

Each entry corresponds to the initial computation time for creating a
NonUniformBSpline object with the specified knot vector. These measurements
capture the full computational cost including recursive Cox-de Boor
evaluations and polynomial constructions.
"""
times_first = []


"""
List storing cached-call execution times for each knot vector configuration.

Each entry corresponds to the retrieval time for accessing a pre-computed
NonUniformBSpline from the registry cache. These measurements demonstrate
the optimal performance achieved through the memoization system, typically
showing order-of-magnitude improvements over first-call times.
"""
times_cached = []


print("Non-uniform B-spline Creation Benchmark")
print("=" * 50)

for i, t in enumerate(knot_vectors):
    """
    Test the second basis function (k=1) across all knot vectors for consistent
    performance comparison. This index is chosen as it typically has non-trivial
    support and represents a common use case in B-spline computations.
    """
    degree = t.degree
    k = 1
    

    """
    Reset the class-level registry before each test iteration to ensure
    clean measurement conditions. This prevents cache contamination
    and guarantees that each first-call measurement represents a true
    computational workload without pre-existing cached results.
    """
    NonUniformBSpline.registry.clear()
    
    
    """
    Measure time for initial NonUniformBSpline construction.
    
    This operation includes:
    - Recursive Cox-de Boor algorithm execution
    - Knot interval analysis and support determination
    - Polynomial coefficient computations
    - Rational function constructions for non-uniform cases
    - Cache storage of the computed result
    """
    start_time = time.time()
    B_first = NonUniformBSpline(t, degree + 1, k)
    first_time = time.time() - start_time


    """
    Measure time for cached NonUniformBSpline retrieval.
    
    This operation demonstrates caching efficiency through:
    - Registry key lookup based on (t.key, degree+1, k) tuple
    - Deep copy of the cached object instance
    - Minimal overhead compared to computational construction
    - O(1) access time complexity in optimal conditions
    """
    start_time = time.time()
    B_cached = NonUniformBSpline(t, degree + 1, k)
    cached_time = time.time() - start_time
    

    """
    Store timing measurements for subsequent analysis.
    
    The significant difference between first_time and cached_time
    quantifies the performance benefit of the caching system
    across different knot vector complexities.
    """
    times_first.append(first_time)
    times_cached.append(cached_time)
    
    """
    Print benchmarking results for immediate performance analysis.
    
    Output includes:
    - Knot vector identifier and polynomial degree
    - Number of knots in the vector
    - First-call execution time in milliseconds
    - Cached-call execution time in milliseconds (typically μs range)
    - Number of objects in registry (verifies caching behavior)
    
    Expected results should show cached calls being significantly faster.
    """
    print(f"Knot Vector {i+1} (Degree {degree}):")
    print(f"  Knots: {len(t.items)} knots")
    print(f"  First call: {first_time*1000:6.2f} ms")
    print(f"  Cached call: {cached_time*1000:6.3f} ms")
    print(f"  Cache hits: {len(NonUniformBSpline.registry)}")
    

Non-uniform B-spline Creation Benchmark
Knot Vector 1 (Degree 2):
  Knots: 6 knots
  First call:  18.21 ms
  Cached call:  0.000 ms
  Cache hits: 6
Knot Vector 2 (Degree 2):
  Knots: 8 knots
  First call:  17.79 ms
  Cached call:  0.000 ms
  Cache hits: 6
Knot Vector 3 (Degree 3):
  Knots: 12 knots
  First call:  29.00 ms
  Cached call:  0.000 ms
  Cache hits: 10
